In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import random
import math
import pickle
import os
import seaborn as sns
import psutil
import statistics
from statistics import mode
from tqdm import tqdm
from scipy.stats import multivariate_hypergeom
from scipy import stats
from sympy.ntheory import factorint
from termcolor import colored, cprint
from scipy.optimize import curve_fit
from sympy import Symbol, Integral
from PIL import Image
from openpyxl import Workbook
wb=Workbook()
ws=wb.active


def gaussian(x,constant,mean,std):
    f=constant*np.exp(-1*(((x-mean)/std)**2)/2)
    return f

def exponential(x,a,b,c,d):
    f=a*np.exp(b*x+c)+d
    return f

def quantile_cauchy(x,gam):
    f=math.tan(math.pi*(x-0.5))*gam
    return f


file=open("prime_set.txt directory","r",encoding='UTF8')#saved_prime_numbers_location
n=file.read()
file.close()
n=n.split(',')
prime_set=[]
for i in n:
    prime_set.append(int(i))

prime_set_initial=prime_set[0:100]


file=open("b_set.txt directory","r",encoding='UTF8')#saved_b_sets_location
n=file.read()
file.close()
n=n.split(',')
b_set=[]
for i in n:
    b_set.append(str(i))

b_set_size=len(b_set)



###############################################################################################################################
###############################################################################################################################
###############################################################################################################################

def N_t(t,N_ori,N_i,grad_var,lag_var):
  pop_num=N_i*math.exp(math.log(N_ori/N_i)*math.exp(-1*math.exp((grad_var*math.exp(1)/math.log(N_ori/N_i))*(lag_var-t)+1)))
  return math.floor(pop_num)


def initial_function_maker(initial_trait_number, prime_set_initial, diverse_s):
  function={}
  color_dict={}
  factor_dict={}
  select_dict={}
  initial_indice_array=[]
  if diverse_s=='on' and initial_trait_number==1:
    function[1]=[1,0]
    factor_dict[1]=b_set[prime_set_initial.index(1)]
    select_dict[1]=0
    color_dict[1]=[1,0,1,0]

  else:
    for i in range(initial_trait_number):
      initial_indice_array.append(prime_set[i])
    for i in initial_indice_array:
      function[i]=[(1/initial_trait_number), 1-(1/initial_trait_number)]
      factor_dict[i]=b_set[prime_set_initial.index(i)]
      select_dict[i]=0
      color_dict[i]="#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])
    
    color_dict[1]='#000000'   

  return function, color_dict, factor_dict, select_dict


def logos(gam, diverse_s, prime_set, generation, function, N, m, s, bound, b_set_size, color_dict, factor_dict, N_i, grad_var,lag_var,dynamic_mode, select_dict):
  prime_sets={}
  prime_set_array=[]

  j=0
  while j in range(len(prime_set)):
    prime_sets[math.floor(j/10000)]=prime_set[j:j+10000]
    j=j+10000
  
  prime_set_array=list(prime_sets.keys())
  history=[]
  color_dict=color_dict
  select_dict=select_dict
  factor_dict=factor_dict
  whole_history={}
  mean_fit_increase_history={}
  prime_index=0
  prime_set_part=copy.deepcopy(prime_sets[prime_set_array[0]])
  prime_index, prime_set_part=initial_function_info(function, prime_set_part, prime_index)
  prime_set_part.sort()
  N_ori=N
  fixation_tracking_array=[]
  first_gen_tracking_array=[generation+1]
  
  for t in range(0,generation+1):
    if dynamic_mode=='on':
        N=N_t(t,N_ori,N_i,grad_var,lag_var)
        
    function, color_dict, factor_dict, prime_index, prime_set_part, prime_set_array, det_for_loop, select_dict = mutation(gam, diverse_s, N, m, s, function, color_dict, factor_dict, b_set_size, prime_index, prime_set_part, prime_set_array, prime_sets, select_dict)
    
    if det_for_loop==-1:
        generation=t
        print("prime_set_shortage for t:",t)
        break
    
    if det_for_loop==-2:
        generation=t
        print("b_set_shortage for t:",t)
        break
    
    function=discriminator(N, function)
    function, mean_fit_increase=potential(function,factor_dict,select_dict,diverse_s,s)
    function=discriminator(N, function)
    function=drift(function, N, N_ori, dynamic_mode)
    function=discriminator(N, function)
      
    for i in function:
      if list(function[i])[0]>=bound:
        if i not in fixation_tracking_array:
          fixation_tracking_array.append([t,i])
    

    if 1 in list(function.keys()):
      if list(function[1])[0]<=bound/2:
        if t<first_gen_tracking_array[0]:
          first_gen_tracking_array.append(t)
    
    whole_history[t]=function
    mean_fit_increase_history[t]=mean_fit_increase

  first_gen_tracking_array.append(generation)
  return generation, whole_history, color_dict, factor_dict, mean_fit_increase_history, prime_set_array, str_to_int(b_set[prime_index]), fixation_tracking_array, first_gen_tracking_array[1]

def initial_function_info(function, prime_set_part, prime_index):
  x=list(function.keys())
  temp_ind=x
  temp_ind.sort()
  prime_index=prime_index+len(temp_ind)-1
  prime_set_part=list(set(prime_set_part)-set(temp_ind))

  return prime_index, prime_set_part

def str_to_int(b_set_element):
  b=b_set_element.replace("b","")
  b=b.replace("'","")
  b=int(b)
  return b

def inverse_permutation(a):
    b = np.arange(a.shape[0])
    b[a] = b.copy()

    return b


def discriminator(N, function):
  x=list(function.keys())
  temp_ind=x
  temp_ind.sort()

  for i in temp_ind:
    y=list(function[i])
    y=y[0]
    if y<(1/N):
      del function[i]


  return function


def mutation(gam, diverse_s, N, m, s, function, color_dict, factor_dict, b_set_size, prime_index, prime_set_part, prime_set_array, prime_sets, select_dict):
    
  x=list(function.keys())
  temp_ind=x
  temp_ind.sort()
  det_for_loop=0
    
  for i in temp_ind:
    y=list(function[i])
    y=y[0]
    if N*m*y>=1:
      exceed_num=math.floor(N*m*y)
      float_num=N*m*y-exceed_num
      function[i]=[y-exceed_num*(1/N),1-(y-exceed_num*(1/N))]
      for j in range(exceed_num):
        
        if prime_set_part==[]:
            
          if prime_set_array==[]:
            det_for_loop=-1
            return function, color_dict, factor_dict, prime_index, prime_set_part, prime_set_array, det_for_loop, select_dict
        
          else:
            prime_set_array.remove(prime_set_array[0])
            prime_set_part=copy.deepcopy(prime_sets[prime_set_array[0]])
            next_prime=prime_set_part[0]
            
        else:
          next_prime=prime_set_part[0]
        
        if b_set_size==prime_index+2:
          det_for_loop=-2
          return function, color_dict, factor_dict, prime_index, prime_set_part, prime_set_array, det_for_loop, select_dict
 
        n=i*next_prime
        function[n]=[1/N,1-(1/N)]
        next_label=b_set[prime_index]
        prime_set_part.remove(next_prime)
        prime_set_part.sort()
        prime_index+=1
        
        color = "#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])
 
        color_dict[n]=color  

        factor_dict[n]=factor_dict[i]+'*'+str(next_label)
        

        if diverse_s=='on':
            rnd=random.uniform(0,1)
            s_val=quantile_cauchy(rnd,gam)
            
            if s_val<0:
              s_val=max(s_val,-2*s)
            if s_val>0:
              s_val=min(s_val,2*s)
            
            temp_num_s=min(abs(s_val/(2*s)),1)
            if s_val<0:
              color_dict[n]=[1-(temp_num_s/2),0,0,temp_num_s]
            if s_val==0:
              color_dict[n]=[1,0,1,0]  
            if s_val>0:
              color_dict[n]=[0,0,1-(temp_num_s/2),temp_num_s]

            
            select_dict[n]=s_val+select_dict[i]

    
      y=list(function[i])[0]
      alpha=1
      if N*y!=0:
        alpha=1-(1/(N*y))

        a=np.array([[1,0],[0,1]])
        b=np.array([[alpha,0],[1-alpha,1]])
        c=np.array([[1,1/N],[0,1-1/N]])

        crit=float_num
        r=random.random()  

        if r<crit:
    
          if prime_set_part==[]:
            
            if prime_set_array==[]:
              det_for_loop=-1
              return function, color_dict, factor_dict, prime_index, prime_set_part, prime_set_array, det_for_loop, select_dict
        
            else:
              prime_set_array.remove(prime_set_array[0])
              prime_set_part=copy.deepcopy(prime_sets[prime_set_array[0]])
              next_prime=prime_set_part[0]  
            
          else:
            next_prime=prime_set_part[0]
        
          if b_set_size==prime_index+2:
            det_for_loop=-2
            return function, color_dict, factor_dict, prime_index, prime_set_part, prime_set_array, det_for_loop, select_dict
        
          n=i*next_prime
          next_label=b_set[prime_index]  
            
          f=np.array(function[i])
          f=b@f
          f=f/np.sum(f)
          f=f.tolist()
          function[i]=f
          function[n]=[0,1]
          g=np.array(function[n])
          g=c@g
          g=g/np.sum(g)
          g=g.tolist()
          function[n]=g
          prime_set_part.remove(next_prime)
          prime_set_part.sort()
          prime_index+=1

          color = "#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])
 
          color_dict[n]=color  

          factor_dict[n]=factor_dict[i]+'*'+str(next_label)

          if diverse_s=='on':
            rnd=random.uniform(0,1)
            s_val=quantile_cauchy(rnd,gam)
            
            if s_val<0:
              s_val=max(s_val,-2*s)
            if s_val>0:
              s_val=min(s_val,2*s)
            
            temp_num_s=min(abs(s_val/(2*s)),1)
            if s_val<0:
              color_dict[n]=[1-(temp_num_s/2),0,0,temp_num_s]
            if s_val==0:
              color_dict[n]=[1,0,1,0]  
            if s_val>0:
              color_dict[n]=[0,0,1-(temp_num_s/2),temp_num_s]
            
            select_dict[n]=s_val+select_dict[i]

            
    else:
      alpha=1
      if N*y!=0:
        alpha=1-(1/(N*y))

      a=np.array([[1,0],[0,1]])
      b=np.array([[alpha,0],[1-alpha,1]])
      c=np.array([[1,1/N],[0,1-1/N]])

      crit=N*m*y
      r=random.random()
    
      if r<crit:
            
        if prime_set_part==[]:
            
          if len(prime_set_array)==1:
            det_for_loop=-1
            return function, color_dict, factor_dict, prime_index, prime_set_part, prime_set_array, det_for_loop, select_dict
        
          else:
            prime_set_array.remove(prime_set_array[0])
            prime_set_part=copy.deepcopy(prime_sets[prime_set_array[0]])
            next_prime=prime_set_part[0]
            
        else:
          next_prime=prime_set_part[0]
        
        if b_set_size==prime_index+2:
          det_for_loop=-2
          return function, color_dict, factor_dict, prime_index, prime_set_part, prime_set_array, det_for_loop, select_dict
        
        n=i*next_prime
        next_label=b_set[prime_index]
        
        f=np.array(function[i])
        f=b@f
        f=f/np.sum(f)
        f=f.tolist()
        function[i]=f
        function[n]=[0,1]
        g=np.array(function[n])
        g=c@g
        g=g/np.sum(g)
        g=g.tolist()
        function[n]=g
        prime_set_part.remove(next_prime)
        prime_set_part.sort()
        prime_index+=1

        color = "#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])

        color_dict[n]=color  

        factor_dict[n]=factor_dict[i]+'*'+str(next_label)
        
        if diverse_s=='on':
            rnd=random.uniform(0,1)
            s_val=quantile_cauchy(rnd,gam)
            
            if s_val<0:
              s_val=max(s_val,-2*s)
            if s_val>0:
              s_val=min(s_val,2*s)
            
            temp_num_s=min(abs(s_val/(2*s)),1)
            if s_val<0:
              color_dict[n]=[1-(temp_num_s/2),0,0,temp_num_s]
            if s_val==0:
              color_dict[n]=[1,0,1,0]  
            if s_val>0:
              color_dict[n]=[0,0,1-(temp_num_s/2),temp_num_s]
            
            select_dict[n]=s_val+select_dict[i]

  return function, color_dict, factor_dict, prime_index, prime_set_part, prime_set_array, det_for_loop, select_dict


def potential(function,factor_dict,select_dict,diverse_s,s):
  temp_fit={}
  temp_pot={}
  temp_rate={}

  x=list(function.keys())
  temp_ind=x
  temp_ind.sort()


  if diverse_s=='on':
    for i in temp_ind:
      temp_fit[i]=1+select_dict[i]
      temp_rate[i]=select_dict[i]
    
  else:
    for i in temp_ind:
      d=factor_dict[i]
      num=d.count('*')
      temp_fit[i]=1+(num*s)
      temp_rate[i]=num*s

  weight=0
  for i in temp_ind:
    y=list(function[i])
    weight=weight+(float(temp_fit[i])*(float(y[0])))

  for i in temp_ind:
    temp_pot[i]=temp_fit[i]/weight

  for i in temp_ind:
    pot=float(temp_pot[i])
    y=list(function[i])

    if pot==1:
      pot_bar=1
    else:
      x=y[0]
      a=(1-pot*x)
      b=(1-x)
      pot_bar=a/b

    A=np.array([[pot,0],[0,pot_bar]])

    y_np=np.array(y)
    y_np=A@y_np
    z=y_np.tolist()
    function[i]=z

  mean_fit_increase=0
  for i in temp_ind:
    y=float(list(function[i])[0])
    mean_fit_increase+=y*temp_rate[i]


  return function, mean_fit_increase


def drift(function, N, N_ori, dynamic_mode): #drift D operator information could be explicitly expressed, but to reduce the computation complexity, only the result after D was expressed here.
  h={}
  ch_function=dict(sorted(function.items()))
  ftns=ch_function.values()
  x=list(function.keys())
  temp_ind=x
  temp_ind.sort()
  length=len(temp_ind)

  for i in temp_ind:
    h[i]=math.floor(float(ch_function[i][0])*N*2)

  allocation_bound=list(h.values())
  
  seed = random.randint(0,2**32-1)
  gen = np.random.Generator(np.random.PCG64(seed))
    
  if dynamic_mode=='on':
    sigmoid_proportion=(1+(1/(math.exp(-1*math.tan(math.pi*(N/N_ori)/2))+1)))/2
    if N/N_ori<0.9:
      randomness=random.uniform(1,1+0.2*math.exp(-1*N/N_ori))
    else:
      randomness=random.uniform(1-0.2*math.exp(-1*math.tan(math.pi*(N/N_ori)/2)),1+0.2*math.exp(-1*math.tan(math.pi*(N/N_ori)/2)))
    
    new_allocation_bound=[]
    for i in allocation_bound:
      new_allocation_bound.append(math.floor(i*randomness*sigmoid_proportion))
    allocation_bound=np.array(new_allocation_bound)
    ordered_values=gen.multivariate_hypergeometric(allocation_bound, N)
    ordered_values=list(ordered_values)
  
  else:
    ordered_values=gen.multivariate_hypergeometric(allocation_bound, N)
    ordered_values=list(ordered_values)

  for i in range(length):
    ordered_values[i]=ordered_values[i]/N

  Y=[]
  for i in range(length):
    Y.append(i)

  for i in range(length):
    Y[i]=[ordered_values[i],1-ordered_values[i]]

  H=dict(zip(temp_ind,(Y)))

  return H



############################################################################################################################
############################################################################################################################
############################################################################################################################




def mat_for_plot(switch_array, whole_history,generation):
  if save_switch_array[3]=='1D' and save_switch_array[4]!=1:
    whole_index=whole_index_finder(whole_history,generation)
    Y=copy.deepcopy(whole_history)
    original_history=copy.deepcopy(Y)
    aligned_history={}
    concatenated_aligned_history={}
    proportion_density_history={}
    new_2D_whole_history={}
    time_interval_dictionary={}
    new_line_whole_history=new_line_whole_history_making_function(whole_history, generation, whole_index)
    
    
  if save_switch_array[3]=='2D' and save_switch_array[4]!=1:
    whole_index=whole_index_finder(whole_history,generation)
    Y=copy.deepcopy(whole_history)
    original_history=copy.deepcopy(Y)
    aligned_history={}
    concatenated_aligned_history={}
    proportion_density_history={}
    new_2D_whole_history={}
    new_line_whole_history={}

    for t in range(generation):
      aligned_history[t]=align_analysis(original_history, aligned_history, t)

    for t in range(generation):
      concatenated_aligned_history[t]=concatenate_function(original_history, aligned_history, t)

    for t in range(generation):
      proportion_density_history[t]=density_giving_function(concatenated_aligned_history, t)
    
    print("pdh done")
    new_2D_whole_history=new_2D_whole_history_making_function(proportion_density_history, generation, whole_index)

    time_interval_dictionary = time_interval(generation, whole_history,whole_index)

    
  if save_switch_array[3]=='both' and save_switch_array[4]!=1:
    whole_index=whole_index_finder(whole_history,generation)
    Y=copy.deepcopy(whole_history)
    original_history=copy.deepcopy(Y)
    aligned_history={}
    concatenated_aligned_history={}
    proportion_density_history={}
    new_2D_whole_history={}
    new_line_whole_history=new_line_whole_history_making_function(whole_history, generation, whole_index)

    for t in range(generation):
      aligned_history[t]=align_analysis(original_history, aligned_history, t)

    for t in range(generation):
      concatenated_aligned_history[t]=concatenate_function(original_history, aligned_history, t)

    for t in range(generation):
      proportion_density_history[t]=density_giving_function(concatenated_aligned_history, t)

    new_2D_whole_history=new_2D_whole_history_making_function(proportion_density_history, generation, whole_index)

    time_interval_dictionary = time_interval(generation, whole_history,whole_index)
  

  return new_line_whole_history, proportion_density_history, new_2D_whole_history, time_interval_dictionary


def concatenate_function(original_history, aligned_history, t):
  A=original_history[t]
  B=list(aligned_history[t])
  C={}

  for element in B:
    C[element]=A[element]

  return C

def density_giving_function(concatenated_aligned_history,t):
  proportion_density={0:0}

  A=concatenated_aligned_history[t].keys()
  temp_set=(list(A))

  for i in concatenated_aligned_history[t]:
    temp_proportion_density={}
    temp_proportion_density[i]=concatenated_aligned_history[t][i][0]
    if temp_set.index(i)==0:
      proportion_density[i]=temp_proportion_density[i]+proportion_density[0]
    else:
      n=temp_set[temp_set.index(i)-1]
      proportion_density[i]=temp_proportion_density[i]+proportion_density[n]

  return proportion_density

def whole_index_finder(whole_history,generation):

  A=set({})
  for t in range(generation):
    A=A.union(set(whole_history[t].keys()))

  whole_index=list(A)
  whole_index.sort()

  return whole_index

def align_algorithm(temp_prior, prior, new_born):

  element_dictionary={}

  for element in new_born:
    A=[]
    for i in prior:
      if element%i==0:
        A.append(i)
    a=max(A)
    element_dictionary[element]=a
    n=temp_prior.index(element_dictionary[element])
    m=temp_prior.index(element)
    temp_prior.insert(n,element)
    del temp_prior[m+1]

  return temp_prior

def align_analysis(original_history, aligned_history, t):

  if t==0:
    A=copy.deepcopy(original_history[0])
    temp_present=list(A.keys())
    temp_present.reverse()
    present=temp_present

    return present

  else:
    A={}
    A=copy.deepcopy(original_history[t])
    current=list(A.keys())

    B={}
    B=copy.deepcopy(aligned_history[t-1])
    prior=list(B)

    new_born=set(current)-set(prior)
    diminished=set(prior)-set(current)

    temp_prior=copy.deepcopy(prior)

    if new_born!=set({}):

      for new in list(new_born):
        temp_prior.append(new)

      temp_present=align_algorithm(temp_prior, prior, new_born)

      if diminished != set({}):
        for old in list(diminished):
          temp_present.remove(old)

        present=temp_present

      else:
        present=temp_present

      return present

    else:
      temp_present=temp_prior
      if diminished != set({}):
        for old in list(diminished):
          temp_present.remove(old)

        present=temp_present

      else:
        present=temp_prior
      return present

def time_interval(generation, whole_history,whole_index):
  time_interval_dict={}

  for i in whole_index:
    time_interval=[]
    for t in range(generation):
      if i in (whole_history[t].keys()):
        time_interval.append(t)
    time_interval_dict[i]=time_interval

  return time_interval_dict

def new_2D_whole_history_making_function(proportion_density_history,generation, whole_index):

  new_2D_whole_history={}
  print("2D_making_ftn started")
  for i in whole_index:
    new_part=[]
    for t in range(generation):
      if i in list(proportion_density_history[t].keys()):
        new_part.append(proportion_density_history[t][i])

    new_2D_whole_history[i]=new_part

  return new_2D_whole_history

def new_line_whole_history_making_function(whole_history, generation, whole_index):

  new_line_whole_history={}
  print("1D_making_ftn started")
  for i in whole_index:
    new_line_part=[]
    for t in range(generation):
      if i in list(whole_history[t].keys()):
        new_line_part.append(float(list((whole_history[t][i]))[0]))

    new_line_whole_history[i]=new_line_part

  return new_line_whole_history


def drawing_one_dim(title, info_with_title, draw_directory_for_iteration_detail, new_line_whole_history, time_interval_dictionary, color_dict, factor_dict, N, m, s, bound, iter):
  plt.rcParams['font.family']='Arial'
  plt.figure(figsize = (15,7))
  plt.xticks(size=15)
  plt.yticks(size=15)
  plt.xlabel('generation',fontsize=25,labelpad=5)  
  plt.ylabel('proportion',fontsize=25,labelpad=20)
  print("actual_1D_started")
  
  for i in new_line_whole_history:
    clr=color_dict[i]

    #plt.plot(time_interval_dictionary[i],new_line_whole_history[i],color=clr,linestyle='-') # could express the border lines in the Muller plot, but not recommended to use
    a=max(new_line_whole_history[i])
    if (a>=bound):
      plt.plot(time_interval_dictionary[i],new_line_whole_history[i],color=clr,linestyle='-',label=factor_dict[i])
    else:
      plt.plot(time_interval_dictionary[i],new_line_whole_history[i],color=clr,linestyle='-')

  plt.savefig(str(draw_directory_for_iteration_detail)+'no title one dim.png')
  plt.rcParams['font.family']='Arial'
  plt.title('one dim with_'+str(info_with_title))
  plt.savefig(str(draw_directory_for_iteration_detail)+'one dim without legend.png')  
  plt.legend(loc=10,fontsize="8")
  plt.savefig(str(draw_directory_for_iteration_detail)+'one dim.png')

  plt.close()
  
def drawing_two_dim(diverse_s, title, info_with_title, draw_directory_for_iteration_detail, new_2D_whole_history, time_interval_dictionary, generation, proportion_density_history, color_dict, N, m, s, iter):
  index_set=list(new_2D_whole_history.keys())

  plt.rcParams['font.family']='Arial'
  plt.figure(figsize = (15,7))
  plt.xticks(size=15)
  plt.yticks(size=15)
  plt.xlabel('generation',fontsize=25,labelpad=5)
  plt.ylabel('proportion',fontsize=25,labelpad=20)
  print("actual_2D_started")


  time_range=[]

  for t in range(generation):
    time_range.append(t)


  for t in range(generation-1):
    temp_ind1=list(proportion_density_history[t].keys())
    temp_ind2=list(proportion_density_history[t+1].keys())

    for i in temp_ind1:
      if i==temp_ind1[-1]:
          break

      else:
        next_ind1=temp_ind1.index(i)+1
        next_key1=temp_ind1[next_ind1]

        if i in temp_ind2:
          if next_key1 in temp_ind2:
            next_ind2=temp_ind2.index(i)+1
            if next_ind2<len(temp_ind2)-1:
              next_key2=temp_ind2[next_ind2]
              plt.fill([time_range[t],time_range[t],time_range[t+1],time_range[t+1]],[proportion_density_history[t][i], proportion_density_history[t][next_key1],proportion_density_history[t+1][next_key2], proportion_density_history[t+1][i]],color=color_dict[next_key1])
            if next_ind2==len(temp_ind2)-1:
              plt.fill([t,t,t+1],[proportion_density_history[t][i],proportion_density_history[t][next_key1],proportion_density_history[t+1][temp_ind2[-2]]],color=color_dict[next_key1])
          else:
            plt.fill([t,t,t+1],[proportion_density_history[t][i],proportion_density_history[t][next_key1],proportion_density_history[t+1][i]],color=color_dict[next_key1])

        else:
          temp_num=0
          while i not in temp_ind2:
            next_ind3=temp_ind1.index(i)-1
            next_key3=temp_ind1[next_ind3]
            i=temp_ind1[next_ind3]
            temp_num=i
          plt.fill([t,t,t+1],[proportion_density_history[t][i],proportion_density_history[t][next_key1],proportion_density_history[t+1][temp_num]],color=color_dict[next_key1])  

  
  plt.rcParams['font.family']='Arial'
  plt.savefig(str(draw_directory_for_iteration_detail)+'no title two dim.png')
  plt.title(str(info_with_title))
  plt.savefig(str(draw_directory_for_iteration_detail)+'two dim.png')

  plt.close()


###############################################################################################################################
###############################################################################################################################
###############################################################################################################################



def mean_velocity_finder(draw_iter_array, save_switch_array, regime, title, info_with_title, directory_for_iteration_detail, mean_fit_increase_history, generation, N, s, m, iter):

  if regime=='concurrent':
    if s/m==1:
        v_theoretical=0.00001
    else:
        v_theoretical=abs((s**2)*(2*math.log(N*s)-math.log(s/m))/(math.log(s/m)**2))
    crit_iter=draw_iter_array[0]
  if regime=='successional':
    v_theoretical=abs((s**2)*N*m)
    crit_iter=draw_iter_array[1]
  if regime=='drift':
    v_theoretical=4*10**(-7)
    crit_iter=draw_iter_array[3]
  if regime=='between':
    v_theoretical=abs((s**2)/(math.log(N*s)+(1/N*m)))
    crit_iter=draw_iter_array[2]
    
    
  mean_fit_increase_history_array=[]
  theoretical_increase_array=[]
  time_array=[]
  for t in range(generation):
    mean_fit_increase_history_array.append(mean_fit_increase_history[t])
    theoretical_increase_array.append(v_theoretical*t)
    time_array.append(t)

  mean_velocity_array=[]
  for t in range(generation):
    temp_mean_velocity=mean_fit_increase_history[t+1]-mean_fit_increase_history[t]
    mean_velocity_array.append(temp_mean_velocity)
    if t==generation-2:
      break

  left_boundary_value=0
  for t in range(generation-2):
    if mean_velocity_array[t]>=v_theoretical*(4/5):
      left_boundary_value=t
      break

  temp_domain=[]  
  for t in range(left_boundary_value,generation-1):
    temp_domain.append(t)

  temp_range=[]
  for t in temp_domain:
    temp_range.append(mean_fit_increase_history[t])
    
  a, b = np.polyfit(temp_domain, temp_range, 1)
  fit_line=[]
  for i in temp_domain:
    fit_line.append(a*i+b)
  

  mean_value=a
  error=round(abs(100*(v_theoretical-mean_value)/v_theoretical),2)

  right_boundary_value=generation-2
  right_function_value=max(fit_line)
  

  velocity_proportion=mean_value/v_theoretical
  if iter<=crit_iter:
    if save_switch_array[0]==0 and save_switch_array[2]==1 and save_switch_array[4]!=1 and save_switch_array[3]=='both' or save_switch_array[3]=='1D' or save_switch_array[3]=='2D':
      draw_directory_for_iteration_detail=str(directory_for_iteration_detail)+str(iter)+'/'
      plt.rcParams['font.family']='Arial'
      plt.figure(figsize = (15,7))
      plt.xticks(size=15)
      plt.yticks(size=15)
      plt.xlabel('generation',fontsize=25,labelpad=5)
      plt.ylabel('mean fitness of generation',fontsize=25,labelpad=20)
      plt.plot(time_array,mean_fit_increase_history_array,color='b',linestyle='-')
      plt.plot(time_array,theoretical_increase_array,color='r',linestyle='-')
      plt.plot(temp_domain,fit_line,color='g',linestyle='--')
    
      os.path.exists(draw_directory_for_iteration_detail) 
      if not os.path.exists(draw_directory_for_iteration_detail):
        os.mkdir(draw_directory_for_iteration_detail)
    
      plt.savefig(str(draw_directory_for_iteration_detail)+'no title velocity, iter_'+str(iter)+'.png')
      plt.rcParams['font.family']='Arial'
      plt.title('finding <s> with '+str(title))
      if s!=0:
        plt.text(0,v_theoretical*generation*(2/5),'v_theoretical='+str(v_theoretical)+'\nv_mean='+str(mean_value)
           +'\nerror='+str(error)+'%'+'\nleft_boundary_value='+str(left_boundary_value)+'\nright_boundary_value='+str(right_boundary_value)
           +'\nleft_function_value='+str(mean_fit_increase_history[left_boundary_value])+'\nright_function_value='+str(right_function_value)
           +'\nvelocity_proportion='+str(velocity_proportion))
        plt.savefig(str(draw_directory_for_iteration_detail)+'title and text velocity, iter_'+str(iter)+'.png')
      else:
        plt.text(0,v_theoretical*generation*(2/5),'v_mean='+str(mean_value)
           +'\nerror='+str(error)+'%'+'\nleft_boundary_value='+str(left_boundary_value)+'\nright_boundary_value='+str(right_boundary_value)
           +'\nleft_function_value='+str(mean_fit_increase_history[left_boundary_value])+'\nright_function_value='+str(right_function_value)
           +'\nvelocity_proportion='+str(velocity_proportion))
        plt.savefig(str(draw_directory_for_iteration_detail)+'title and text velocity, iter_'+str(iter)+'.png')  

    plt.close()

  return velocity_proportion, v_theoretical



def distribution_data_ftn_1(N, s, velocity_proportion_array, iteration_size, vel_draw_left_criteria, vel_draw_right_criteria, regime):
    
    if regime=='concurrent':
      vel_draw_left_cri=vel_draw_left_criteria[0]
      vel_draw_right_cri=vel_draw_right_criteria[0] 
    
    if regime=='successional':
      vel_draw_left_cri=vel_draw_left_criteria[1]
      vel_draw_right_cri=vel_draw_right_criteria[1]
        
    if regime=='between':
      vel_draw_left_cri=vel_draw_left_criteria[2]
      vel_draw_right_cri=vel_draw_right_criteria[2]    
    
    if regime=='drift':
      vel_draw_left_cri=vel_draw_left_criteria[3]
      vel_draw_right_cri=vel_draw_right_criteria[3]
   
    
    bin_array=[math.floor(iteration_size/20),math.floor(iteration_size/18),math.floor(iteration_size/15),math.floor(iteration_size/12),math.floor(iteration_size/10),math.floor(iteration_size/8),math.floor(iteration_size/5),math.floor(iteration_size/4),math.floor(iteration_size/3),math.floor(iteration_size/2)]
    
    statistics_dictionary_g={}  
    statistics_dictionary_e={}
    for b in bin_array:  
      temp_X=list(np.histogram(velocity_proportion_array,bins=b)[1])
      temp_X.pop(-1)
      Y=np.histogram(velocity_proportion_array,bins=b)[0]
    
      temp_mean_temp_X=statistics.mean(temp_X)
      temp_stdev_temp_X=statistics.stdev(temp_X)

      temp_modi_X=[]
      if any(t_x < 0 for t_x in temp_X):
        for t_x in temp_X:
          if t_x>=-1*vel_draw_left_cri*temp_stdev_temp_X+temp_mean_temp_X:
            temp_modi_X.append(t_x)    
    
      else:
        for t_x in temp_X:
          if t_x>=max(0.000000000001, -1*vel_draw_left_cri*temp_stdev_temp_X+temp_mean_temp_X):
            temp_modi_X.append(t_x)       
    
      temp_mean_modi_X=statistics.mean(temp_modi_X)
      temp_stdev_modi_X=statistics.stdev(temp_modi_X)
  
      modi_X=[]
      for t_x in temp_modi_X:
        if t_x<=vel_draw_right_cri*temp_stdev_modi_X+temp_mean_modi_X:
          modi_X.append(t_x)
          
    
      min_modi_X_ind=(temp_X.index(min(modi_X)))
      max_modi_X_ind=(temp_X.index(max(modi_X)))
      modi_X=np.array(modi_X)
      
   
      modi_Y=[]
      for i in range(len(Y)):
        if min_modi_X_ind<=i<=max_modi_X_ind:
          modi_Y.append(Y[i])
      
      modi_Y=np.array(modi_Y)   
      
      mean=statistics.mean(modi_X)
      stdev=statistics.stdev(modi_X)
      initial_guess=[max(list(modi_Y)),mean,stdev]
      
      modi_velocity_array=[]
      for i in range(len(velocity_proportion_array)):
        if min(modi_X)<=velocity_proportion_array[i]<=max(modi_X):
          modi_velocity_array.append(velocity_proportion_array[i])  
      
      modi_velocity_array=np.array(modi_velocity_array)
    
      
      if s==0 or stdev/mean>=1:
        return [-1,-1,-1,-1,-1,-1,-1,-1]
    
      else:
        popt_g, pcov_g=curve_fit(gaussian,modi_X,modi_Y, maxfev=1000000, p0=initial_guess,
                             bounds=([4*max(list(modi_Y))/5,mean-stdev/2,stdev/2],[6*max(list(modi_Y))/5,mean+stdev/2,stdev*3/2]))
        stat_result_g = stats.kstest(modi_Y, gaussian(modi_X,popt_g[0],popt_g[1],popt_g[2]))
        p_value_g=stat_result_g[1]
        statistics_dictionary_g[b]=[p_value_g,mean,stdev,b,popt_g,modi_X,modi_Y,modi_velocity_array]
        if p_value_g<0.05 and regime!='between':
          popt_e, pcov_e=curve_fit(exponential,modi_X,modi_Y, maxfev=1000000,
                           bounds=([4*max(list(modi_Y))/5,-np.inf,-np.inf,0],[6*max(list(modi_Y))/5,np.inf, np.inf, np.inf]))
          stat_result_e= stats.kstest(modi_Y, exponential(modi_X,popt_e[0],popt_e[1],popt_e[2],popt_e[3]))
          p_value_e=stat_result_e[1]
          statistics_dictionary_e[b]=[p_value_e,mean,stdev,b,popt_e,modi_X,modi_Y,modi_velocity_array]
        else: 
          statistics_dictionary_e[b]=[0,0,0,0,0,0,0,0]
    
    p_value_array_g=[]
    p_value_array_e=[]
    for b in bin_array:
      p_value_array_g.append(statistics_dictionary_g[b][0])
      p_value_array_e.append(statistics_dictionary_e[b][0])
    
    max_p_g=max(p_value_array_g)
    max_p_e=max(p_value_array_e)
    
    if max_p_g>max_p_e:
      max_p=max_p_g
      p_value_array=p_value_array_g
      statistics_dictionary=statistics_dictionary_g
    else:
      max_p=max_p_e
      p_value_array=p_value_array_e
      statistics_dictionary=statistics_dictionary_e
        
    b_info=bin_array[p_value_array.index(max_p)]

    info_for_the_distribution_1=statistics_dictionary[b_info]

    return info_for_the_distribution_1


def draw_for_the_distribution_1(info_for_the_distribution_1, velocity_proportion_array, directory_for_total, title, info_with_title, iteration_size, N, m, s, regime, v_theoretical):
    if info_for_the_distribution_1[0]<0:
        print("skip for",info_with_title)
        return 0, 0, 0, 0, 0, 0, 0, 0
    
    else:
      vel_array_for_stat=[]  
      modi_X=info_for_the_distribution_1[5]  
      for vel in velocity_proportion_array:
        if min(list(modi_X))<=vel<=max(list(modi_X)):
          vel_array_for_stat.append(vel)  
        
      new_mean=statistics.mean(vel_array_for_stat)
      new_stdev=statistics.stdev(vel_array_for_stat)
      p_value=info_for_the_distribution_1[0]
      mean=info_for_the_distribution_1[1]
      stdev=info_for_the_distribution_1[2]
      b=info_for_the_distribution_1[3]
      popt=info_for_the_distribution_1[4]
      modi_Y=info_for_the_distribution_1[6]
      modi_velocity_array=info_for_the_distribution_1[7]
      
      if save_switch_array[0]==0 and save_switch_array[2]==1 and save_switch_array[4]!=1:
        plt.figure(figsize = (10,10))
        plt.rcParams['font.family']='Arial'
        plt.xticks(size=15)
        plt.yticks(size=15)
        plt.ylabel("count",fontsize=25,labelpad=20)
        plt.xlabel("proportion for beneficial muation accumulation",fontsize=25,labelpad=5)
        plt.hist(velocity_proportion_array,bins=modi_X,color='skyblue',edgecolor='black')
        plt.plot(modi_X,gaussian(modi_X,popt[0],popt[1],popt[2]),color='darkblue')
        plt.savefig(str(directory_for_total)+'without_legend, velocity_proportion.png')
        if s!=0:
          plt.text((max(modi_X)+min(modi_X))/2,(max(modi_Y)+min(modi_Y))/4,'mean='+str(new_mean)+'\nstdev='+str(abs(new_stdev))+
               '\np_value='+str(p_value)+'\n'+str(info_with_title)+
               '\nbin_size='+str(b)+'\nitr='+str(iteration_size)+
               '\nregime='+str(regime)+'\nv_th='+str(v_theoretical)+"\nrepresented_y="+str(sum(modi_Y))+
               '\nmax_modi_Y='+str(max(list(modi_Y))),color='r')
        else:
          plt.text((max(modi_X)+min(modi_X))/2,(max(modi_Y)+min(modi_Y))/4,'mean='+str(new_mean)+'\nstdev='+str(abs(new_stdev))+
               '\np_value='+str(p_value)+'\n'+str(info_with_title)+
               '\nbin_size='+str(b)+'\nitr='+str(iteration_size)+
               '\nregime='+str(regime)+'\nv_th='+str(v_theoretical)+"\nrepresented_y="+str(sum(modi_Y))+
               '\nmax_modi_Y='+str(max(list(modi_Y))),color='r')  
            
        plt.savefig(str(directory_for_total)+'velocity_proportion.png')
        plt.close()  
        
        plt.rcParams['font.family']='Arial'
        plt.figure(figsize = (10,10))
        sns.displot(modi_velocity_array, bins=modi_X, kde='True',color='darkblue',facecolor='skyblue').fig.set_size_inches(10,10)
        plt.xticks(size=15)
        plt.yticks(size=15)
        plt.ylabel("count",fontsize=25,labelpad=20)
        plt.xlabel("proportion for beneficial muation rate",fontsize=25,labelpad=5)  
        plt.savefig(str(directory_for_total)+'kde_velocity_proportion.png')
      
        plt.close()

    return new_mean, new_stdev, p_value, b        
        
##############################################################################################################################
##############################################################################################################################      
##############################################################################################################################


def fixation_information(last_b, fixation_tracking_array, factor_dict):
  
  param_max_t=0
  param_ind=0
  for x in range(len(fixation_tracking_array)):
    if fixation_tracking_array[x][0]>param_max_t:
      param_max_t=fixation_tracking_array[x][0]
      param_ind=fixation_tracking_array[x][1] 
    
  fixation_probability=(factor_dict[param_ind].count('*')+1)/last_b  

  return fixation_probability

def fixation_proportion_finder(regime, N, s, m, alp_array,fixation_probability):
  if regime=='concurrent':
    possibility_array=[]
    theoretical_array=[]
    error_array=[]
    
    for alp in alp_array:
      if 4*s*math.exp((-1*m*N*math.log(N)*math.exp(-1*alp*s)*4*(s+1/alp))/s)>10**(-5):
        possibility_array.append(alp)
        theoretical_array.append(4*s*math.exp((-1*m*N*math.log(N)*math.exp(-1*alp*s)*4*(s+1/alp))/s))
    
    for th in theoretical_array:
      err=abs(th-fixation_probability)/th
      error_array.append(err)
    
    min_err_ind=error_array.index(min(error_array))
    the_alp=possibility_array[min_err_ind]
    fixation_p_theoretical=theoretical_array[min_err_ind]
    
  if regime=='successional':
    fixation_p_theoretical=(1-math.exp(-4*s))/(1-math.exp(-4*N*s))
    the_alp=0
  if regime=='drift':
    if s==0:
      fixation_p_theoretical=1/N
    else:
      fixation_p_theoretical=(1-math.exp(-4*s))/(1-math.exp(-4*N*s))*0.001  
    the_alp=0
  if regime=='between':
    fixation_p_theoretical=1
    the_alp=0
    
  fixation_proportion=fixation_probability/fixation_p_theoretical  
  return fixation_proportion, fixation_p_theoretical,the_alp

  

def distribution_data_ftn_2(N, fixation_proportion_array, iteration_size, fix_draw_left_criteria, fix_draw_right_criteria, regime):
    
    if regime=='concurrent':
      fix_draw_left_cri=fix_draw_left_criteria[0]
      fix_draw_right_cri=fix_draw_right_criteria[0] 
    
    if regime=='successional':
      fix_draw_left_cri=fix_draw_left_criteria[1]
      fix_draw_right_cri=fix_draw_right_criteria[1] 
        
    if regime=='between':
      fix_draw_left_cri=fix_draw_left_criteria[2]
      fix_draw_right_cri=fix_draw_right_criteria[2] 
        
    if regime=='drift':
      fix_draw_left_cri=fix_draw_left_criteria[3]
      fix_draw_right_cri=fix_draw_right_criteria[3]    
    
    bin_array=[math.floor(iteration_size/20),math.floor(iteration_size/18),math.floor(iteration_size/15),math.floor(iteration_size/12),math.floor(iteration_size/10),math.floor(iteration_size/8),math.floor(iteration_size/5),math.floor(iteration_size/4),math.floor(iteration_size/3),math.floor(iteration_size/2)]    
    
    statistics_dictionary_g={}  
    statistics_dictionary_e={}
    for b in bin_array:  
      temp_X=list(np.histogram(fixation_proportion_array,bins=b)[1])
      temp_X.pop(-1)
      Y=np.histogram(fixation_proportion_array,bins=b)[0]

      temp_mean_temp_X=statistics.mean(temp_X)
      temp_stdev_temp_X=statistics.stdev(temp_X)
    
      temp_modi_X=[]
      for t_x in temp_X:
        if t_x>=max(0.000000001,-1*fix_draw_left_cri*temp_stdev_temp_X+temp_mean_temp_X):
          temp_modi_X.append(t_x)  
    
      temp_mean_modi_X=statistics.mean(temp_modi_X)
      temp_stdev_modi_X=statistics.stdev(temp_modi_X)

      modi_X=[]
      for t_x in temp_modi_X:
        if t_x<=fix_draw_right_cri*temp_stdev_modi_X+temp_mean_modi_X:
          modi_X.append(t_x)
          
    
      min_modi_X_ind=(temp_X.index(min(modi_X)))
      max_modi_X_ind=(temp_X.index(max(modi_X)))
      modi_X=np.array(modi_X)
      
   
      modi_Y=[]
      for i in range(len(Y)):
        if min_modi_X_ind<=i<=max_modi_X_ind:
          modi_Y.append(Y[i])
      
      modi_Y=np.array(modi_Y)   
      
      mean=statistics.mean(modi_X)
      stdev=statistics.stdev(modi_X)
      initial_guess=[max(list(modi_Y)),mean,stdev]
      
      modi_fixation_array=[]
      for i in range(len(fixation_proportion_array)):
        if min(modi_X)<=fixation_proportion_array[i]<=max(modi_X):
          modi_fixation_array.append(fixation_proportion_array[i])  
      
      modi_fixation_array=np.array(modi_fixation_array)
    
      
      if stdev/mean>=1:
        return [-1,-1,-1,-1,-1,-1,-1,-1]
    
      else:
        popt_g, pcov_g=curve_fit(gaussian,modi_X,modi_Y, maxfev=1000000, p0=initial_guess,
                             bounds=([4*max(list(modi_Y))/5,mean-stdev/2,stdev/2],[6*max(list(modi_Y))/5,mean+stdev/2,stdev*3/2]))
        stat_result_g = stats.kstest(modi_Y, gaussian(modi_X,popt_g[0],popt_g[1],popt_g[2]))
        p_value_g=stat_result_g[1]
        statistics_dictionary_g[b]=[p_value_g,mean,stdev,b,popt_g,modi_X,modi_Y,modi_fixation_array]

    
    p_value_array_g=[]

    for b in bin_array:
      p_value_array_g.append(statistics_dictionary_g[b][0])
 
    
    max_p_g=max(p_value_array_g)

    

    max_p=max_p_g
    p_value_array=p_value_array_g
    statistics_dictionary=statistics_dictionary_g

        
    b_info=bin_array[p_value_array.index(max_p)]

    info_for_the_distribution_2=statistics_dictionary[b_info]

    return info_for_the_distribution_2


def draw_for_the_distribution_2(final_alp, info_for_the_distribution_2, fixation_proportion_array, directory_for_total, title, info_with_title, iteration_size, N, m, s, regime, fixation_p_theoretical):
    if info_for_the_distribution_2[0]<0:
        print("skip for",info_with_title)
        return 0, 0, 0, 0, 0, 0, 0, 0
    
    else:
      fix_array_for_stat=[]  
      modi_X=info_for_the_distribution_2[5]  
      for fix in fixation_proportion_array:
        if min(list(modi_X))<=fix<=max(list(modi_X)):
          fix_array_for_stat.append(fix)  
        
      new_mean=statistics.mean(fix_array_for_stat)
      new_stdev=statistics.stdev(fix_array_for_stat)
      p_value=info_for_the_distribution_2[0]
      mean=info_for_the_distribution_2[1]
      stdev=info_for_the_distribution_2[2]
      b=info_for_the_distribution_2[3]
      popt=info_for_the_distribution_2[4]
      modi_Y=info_for_the_distribution_2[6]
      modi_velocity_array=info_for_the_distribution_2[7]
      
      if save_switch_array[0]==0 and save_switch_array[2]==1 and save_switch_array[4]!=1:
        plt.figure(figsize = (10,10))
        plt.rcParams['font.family']='Arial'
        plt.xticks(size=15)
        plt.yticks(size=15)
        plt.ylabel("count",fontsize=25,labelpad=20)
        plt.xlabel("proportion for fixation proportion",fontsize=25,labelpad=5)
        plt.hist(fixation_proportion_array,bins=modi_X,color='greenyellow',edgecolor='black')
        plt.plot(modi_X,gaussian(modi_X,popt[0],popt[1],popt[2]),color='darkgreen')
        plt.savefig(str(directory_for_total)+'without_legend, velocity_proportion.png')
        if s!=0:
          plt.text((max(modi_X)+min(modi_X))/2,(max(modi_Y)+min(modi_Y))/4,'mean='+str(new_mean)+'\nstdev='+str(abs(new_stdev))+
               '\np_value='+str(p_value)+'\n'+str(info_with_title)+
               '\nbin_size='+str(b)+'\nitr='+str(iteration_size)+
               '\nregime='+str(regime)+'\nfixation_p_th='+str(fixation_p_theoretical)+"\nrepresented_y="+str(sum(modi_Y))+
               '\nmax_modi_Y='+str(max(list(modi_Y))),color='r')
   
    
        else:
          plt.text((max(modi_X)+min(modi_X))/2,(max(modi_Y)+min(modi_Y))/4,'mean='+str(new_mean)+'\nstdev='+str(abs(new_stdev))+
               '\np_value='+str(p_value)+'\n'+str(info_with_title)+
               '\nbin_size='+str(b)+'\nitr='+str(iteration_size)+
               '\nregime='+str(regime)+'\nfixation_p_th='+str(fixation_p_theoretical)+"\nrepresented_y="+str(sum(modi_Y))+
               '\nmax_modi_Y='+str(max(list(modi_Y))),color='r')  
            
        plt.savefig(str(directory_for_total)+'fixation_proportion.png')
        plt.close()  
      
        
        
        plt.figure(figsize = (10,10))
        plt.rcParams['font.family']='Arial'
        sns.displot(modi_velocity_array, bins=modi_X, kde='True',color='darkgreen',facecolor='greenyellow').fig.set_size_inches(10,10)
        plt.xticks(size=15)
        plt.yticks(size=15)
        plt.ylabel("count",fontsize=25,labelpad=20)
        plt.xlabel("proportion for fixation rate",fontsize=25,labelpad=5)  
        
        plt.savefig(str(directory_for_total)+'kde_fixation_proportion.png')
 
        plt.close()

    return new_mean, new_stdev, p_value, b
 

#############################################################################################################################
#############################################################################################################################
#############################################################################################################################



def information_from_logos(gam, diverse_s, draw_iter_array, alp_array, save_switch_array, directory_for_total, directory_for_iteration_detail, regime, title, info_with_title, directory, prime_set, generation,function,N,m,s,bound, b_set_size, iter, color_dict, factor_dict, N_i, grad_var, lag_var, dynamic_mode, select_dict):      
  
  if regime=='concurrent':
    crit_iter=draw_iter_array[0]    
  if regime=='successional':
    crit_iter=draw_iter_array[1]    
  if regime=='between':
    crit_iter=draw_iter_array[2]    
  if regime=='drift':
    crit_iter=draw_iter_array[3]      

  save_switch_array[1]=1
  logos_data= logos(gam, diverse_s, prime_set, generation, function, N, m, s, bound, b_set_size, color_dict, factor_dict,N_i,grad_var,lag_var,dynamic_mode,select_dict)
  switch_for_save(save_switch_array, directory_for_iteration_detail, logos_data, 'logos_data', iter)
  
  generation=logos_data[0]
  whole_history=logos_data[1]
  color_dict=logos_data[2]
  factor_dict=logos_data[3]
  mean_fit_increase_history=logos_data[4]
  prime_set_array=logos_data[5]
  last_b=logos_data[6]
  fixation_tracking_array=logos_data[7]
  first_gen=logos_data[8]

  mean_velocity_finder_data=mean_velocity_finder(draw_iter_array, save_switch_array, regime, title, info_with_title, directory_for_iteration_detail, mean_fit_increase_history, generation, N, s, m, iter)
  switch_for_save(save_switch_array, directory_for_iteration_detail, mean_velocity_finder_data, 'mean_velocity_finder_data', iter)
    
  velocity_proportion=mean_velocity_finder_data[0]
  v_theoretical=mean_velocity_finder_data[1]  

  fixation_probability=fixation_information(last_b,fixation_tracking_array, factor_dict)
  fixation_proportion_finder_data=fixation_proportion_finder(regime, N, s, m, alp_array,fixation_probability)
  
  fixation_proportion=fixation_proportion_finder_data[0]
  fixation_p_theoretical=fixation_proportion_finder_data[1]
  the_alp=fixation_proportion_finder_data[2]
  
    
  if iter<=crit_iter and save_switch_array[0]==0 and save_switch_array[2]==1 and save_switch_array[3]=='both' and save_switch_array[4]!=1:
    draw_directory_for_iteration_detail=str(directory_for_iteration_detail)+str(iter)+'/'
    
    mat_for_plot_data= mat_for_plot(save_switch_array, whole_history,generation)
    switch_for_save(save_switch_array, directory_for_iteration_detail, mat_for_plot_data, 'mat_for_plot_data', iter)
  
    new_line_whole_history=mat_for_plot_data[0]
    proportion_density_history=mat_for_plot_data[1]
    new_2D_whole_history=mat_for_plot_data[2]
    time_interval_dictionary=mat_for_plot_data[3]
    print("mat_for-plot done")
    
    drawing_one_dim(title, info_with_title, draw_directory_for_iteration_detail, new_line_whole_history, time_interval_dictionary, color_dict, factor_dict, N, m, s, bound, iter)
    print("one_dim_done")
    
    drawing_two_dim(diverse_s, title, info_with_title, draw_directory_for_iteration_detail, new_2D_whole_history, time_interval_dictionary, generation, proportion_density_history, color_dict, N, m, s, iter)
    print("two_dim_done")
    
  if iter<=crit_iter and save_switch_array[0]==0 and save_switch_array[2]==1 and save_switch_array[3]=='1D' and save_switch_array[4]!=1:
    draw_directory_for_iteration_detail=str(directory_for_iteration_detail)+str(iter)+'/'
    
    mat_for_plot_data= mat_for_plot(save_switch_array, whole_history,generation)
    switch_for_save(save_switch_array, directory_for_iteration_detail, mat_for_plot_data, 'mat_for_plot_data', iter)
  
    new_line_whole_history=mat_for_plot_data[0]
    print("mat_for-plot done")
    
    drawing_one_dim(title, info_with_title, draw_directory_for_iteration_detail, new_line_whole_history, time_interval_dictionary, color_dict, factor_dict, N, m, s, bound, iter)
    print("one_dim_done")
    
  if iter<=crit_iter and save_switch_array[0]==0 and save_switch_array[2]==1 and save_switch_array[3]=='2D' and save_switch_array[4]!=1:
    draw_directory_for_iteration_detail=str(directory_for_iteration_detail)+str(iter)+'/'
    
    mat_for_plot_data= mat_for_plot(save_switch_array, whole_history,generation)
    switch_for_save(save_switch_array, directory_for_iteration_detail, mat_for_plot_data, 'mat_for_plot_data', iter)

    proportion_density_history=mat_for_plot_data[1]
    new_2D_whole_history=mat_for_plot_data[2]
    time_interval_dictionary=mat_for_plot_data[3]
    print("mat_for-plot done")
    
    drawing_two_dim(diverse_s, title, info_with_title, draw_directory_for_iteration_detail, new_2D_whole_history, time_interval_dictionary, generation, proportion_density_history, color_dict, N, m, s, iter)
    print("two_dim_done")    




  return velocity_proportion, v_theoretical, fixation_proportion, fixation_p_theoretical, the_alp, first_gen


###############################################################################################################################
###############################################################################################################################
###############################################################################################################################


def switch_for_save(save_switch_array, directory, obj, obj_name, iter): #switch array determines 1. to turn on the save, 2. to reflect iteration or not

    if save_switch_array[0]==0:
        return
    else:
        if save_switch_array[1]==0: #total values
            with open(str(directory)+str(obj_name)+'.pkl','wb') as fp:
                pickle.dump(obj, fp) 
        if save_switch_array[1]==1: #specific iter values
            with open(str(directory)+str(obj_name)+'/'+str(iter)+'.pkl','wb') as fp:
                pickle.dump(obj, fp) 

def directory_making_ftn(save_switch_array, directory, title, info_with_title):
    
    if save_switch_array[0]==0:
        if save_switch_array[2]==1:
          new_directory=str(directory)+str(title)+'/'
          os.mkdir(new_directory)
          f = open(str(new_directory)+'core_variables.txt', 'w')
          with open(str(new_directory)+'core_variables.txt', 'w') as f:
            f.write(str(info_with_title))
          f.close()  
          directory_for_total=str(new_directory)+'total/'
          os.mkdir(directory_for_total)
          directory_for_iteration_detail=str(new_directory)+'iteration_detail/'
          os.mkdir(directory_for_iteration_detail)

        else:
          new_directory=str(directory)+str(title)+'/'  
          directory_for_total=str(new_directory)+'total/'
          directory_for_iteration_detail=str(new_directory)+'iteration_detail/'  
            
        return new_directory, directory_for_total, directory_for_iteration_detail
    
    else:
        new_directory=str(directory)+str(title)+'/'
        directory_for_total=str(new_directory)+'total/'
        directory_for_iteration_detail=str(new_directory)+'iteration_detail/'
        os.mkdir(new_directory) 
        os.mkdir(directory_for_total)
        os.mkdir(str(directory_for_iteration_detail))
        os.mkdir(str(directory_for_iteration_detail)+'logos_data/')
        os.mkdir(str(directory_for_iteration_detail)+'mean_velocity_finder_data/')
        os.mkdir(str(directory_for_iteration_detail)+'mat_for_plot_data/')
        return new_directory, directory_for_total, directory_for_iteration_detail
    

def alp_array_maker(alp_condition_array):
  alp_array=list(np.arange(alp_condition_array[0], alp_condition_array[1]+1,step=alp_condition_array[2]))
  return alp_array    



##########################################################################################################################   
##########################################################################################################################
##########################################################################################################################

    
def statistical_info(par_s, diverse_s, draw_iter_array, training_size, alp_array, save_switch_array, iteration_size, core_variable_array, directory, prime_set, prime_set_initial, bound, b_set_size, vel_draw_left_criteria, vel_draw_right_criteria, fix_draw_left_criteria, fix_draw_right_criteria,N_i,grad_var,lag_var,dynamic_mode):
  for i in range(len(core_variable_array)):
    N=core_variable_array[i][0]
    m=core_variable_array[i][1]
    s=core_variable_array[i][2]
    generation=core_variable_array[i][3]
    initial_trait_number=core_variable_array[i][4]
    regime=core_variable_array[i][5]
    gam=par_s*s
    
    if diverse_s=='on':
      title = 'N_'+str("{:.2e}".format(N))+', m_'+str("{:.2e}".format(m))+', delta s_'+str("{:.2e}".format(s))+', g_'+str(generation)+', i_t_n_'+str(initial_trait_number)+', itr_'+str(iteration_size)+', regime_'+str(regime)+', gamma_'+str("{:.2e}".format(gam))
      info_with_title='N_'+str("{:.2e}".format(N))+', m_'+str("{:.2e}".format(m))+', delta s_'+str("{:.2e}".format(s))+',\n g_'+str(generation)+', initial_n_'+str(initial_trait_number)+', itr_'+str(iteration_size)+', regime_'+str(regime)+', gamma_'+str("{:.2e}".format(gam))
    else:
      title = 'N_'+str("{:.2e}".format(N))+', m_'+str("{:.2e}".format(m))+', delta s_'+str("{:.2e}".format(s))+', g_'+str(generation)+', i_t_n_'+str(initial_trait_number)+', itr_'+str(iteration_size)+', regime_'+str(regime)
      info_with_title='N_'+str("{:.2e}".format(N))+', m_'+str("{:.2e}".format(m))+', delta s_'+str("{:.2e}".format(s))+',\n g_'+str(generation)+', initial_n_'+str(initial_trait_number)+', itr_'+str(iteration_size)+', regime_'+str(regime)

    new_directory, directory_for_total, directory_for_iteration_detail=directory_making_ftn(save_switch_array, directory, title, info_with_title)
    
    velocity_proportion_array=[]
    fixation_proportion_array=[]
    the_alp_array=[]
    final_alp=0
    first_gen_array=[]

    function, color_dict, factor_dict, select_dict = initial_function_maker(initial_trait_number, prime_set_initial, diverse_s)
        
    for iter in tqdm(range(iteration_size)):
      if iter<=training_size:         
        information_from_logos_data=information_from_logos(gam, diverse_s, draw_iter_array, alp_array, save_switch_array, directory_for_total, directory_for_iteration_detail, regime, title, info_with_title, directory, prime_set, generation,function,N,m,s,bound, b_set_size, iter, color_dict, factor_dict, N_i, grad_var, lag_var, dynamic_mode, select_dict)
        velocity_proportion=information_from_logos_data[0]
        v_theoretical=information_from_logos_data[1]
        the_alp=information_from_logos_data[4]
        first_gen=information_from_logos_data[5]
        velocity_proportion_array.append(velocity_proportion)
        the_alp_array.append(the_alp)
        first_gen_array.append(first_gen)
      
      if iter>=training_size:
        final_alp=statistics.mean(the_alp_array)
        information_from_logos_data=information_from_logos(gam, diverse_s, draw_iter_array, [final_alp], save_switch_array, directory_for_total, directory_for_iteration_detail, regime, title, info_with_title, directory, prime_set, generation,function,N,m,s,bound, b_set_size, iter, color_dict, factor_dict, N_i, grad_var, lag_var, dynamic_mode, select_dict)
        velocity_proportion=information_from_logos_data[0]
        v_theoretical=information_from_logos_data[1]
        fixation_proportion=information_from_logos_data[2]
        fixation_p_theoretical=information_from_logos_data[3]
        first_gen=information_from_logos_data[5]
        velocity_proportion_array.append(velocity_proportion)  
        fixation_proportion_array.append(fixation_proportion)
        first_gen_array.append(first_gen)
    
    save_switch_array[1]==0
    switch_for_save(save_switch_array, directory_for_total, velocity_proportion_array, 'velocity_proportion_array', -1)
   
    info_for_the_distribution_1=distribution_data_ftn_1(N, s, velocity_proportion_array, iteration_size, vel_draw_left_criteria, vel_draw_right_criteria, regime)
  
    switch_for_save(save_switch_array, directory_for_total,info_for_the_distribution_1, 'info_for_the_distribution_1', -1)
           
    id1=draw_for_the_distribution_1(info_for_the_distribution_1, velocity_proportion_array, directory_for_total, title, info_with_title, iteration_size, N, m, s, regime, v_theoretical)

            
    
    
    switch_for_save(save_switch_array, directory_for_total, fixation_proportion_array, 'fixation_proportion_array', -1)
   
    info_for_the_distribution_2=distribution_data_ftn_2(N, fixation_proportion_array, iteration_size, fix_draw_left_criteria, fix_draw_right_criteria, regime)
  
    switch_for_save(save_switch_array, directory_for_total,info_for_the_distribution_2, 'info_for_the_distribution_2', -1)
           
    id2=draw_for_the_distribution_2(final_alp, info_for_the_distribution_2, fixation_proportion_array, directory_for_total, title, info_with_title, iteration_size, N, m, s, regime, fixation_p_theoretical)       
    
    #mean_first_gen=statistics.mean(first_gen_array) # gives crude sense for when the first fixation takes place, but it is unreliable: used only to approximately get sense
    
    # info_for_draw_for_the_distribution_1=new_mean, new_stdev, p_value, mean, stdev, b, popt
    if diverse_s=='on':    
      if s!=0:
        if s==m:
          xl_array=["s/m=1",generation,N,s,m,regime,gam,"/",id1[0],id1[1],id1[2],id1[3],"/",id2[0],id2[1],id2[2],id2[3],final_alp]
        else:
          xl_array=["s/m fine",generation,N,s,m,regime,gam,"/",id1[0],id1[1],id1[2],id1[3],"/",id2[0],id2[1],id2[2],id2[3],final_alp]
      if s==0:
        xl_array=["s=0",generation,N,s,m,regime,gam,"/",id1[0],id1[1],id1[2],id1[3],"/",id2[0],id2[1],id2[2],id2[3],'none']  
    
    else:
      if s!=0:
        if s==m:
          xl_array=["s/m=1",generation,N,s,m,regime,"/",id1[0],id1[1],id1[2],id1[3],"/",id2[0],id2[1],id2[2],id2[3],final_alp]
        else:
          xl_array=["s/m fine",generation,N,s,m,regime,"/",id1[0],id1[1],id1[2],id1[3],"/",id2[0],id2[1],id2[2],id2[3],final_alp]
      if s==0:
        xl_array=["s=0",generation,N,s,m,regime,"/",id1[0],id1[1],id1[2],id1[3],"/",id2[0],id2[1],id2[2],id2[3],'none']  

    ws.append(xl_array)
    wb.save(str(directory)+"total parameters1.xlsx")  
    try:
        ram_info = psutil.virtual_memory()
        print(f"Total: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
        print(f"Available: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
        print(f"Used: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
        print(f"Percentage usage: {ram_info.percent}%")
    except FileNotFoundError:
        print("Ram info not available on this system")

       

##################################################################################################################################       
##################################################################################################################################    
##################################################################################################################################



def core_variable_array_maker(n_array, m_array, s_array, g_array, i_array, regime_array):

  core_variable_array=[]
  
  for N in n_array:
    #if 10**6>N>=10**5:
      for m in m_array:
        for s in s_array:
          if s!=0:
            if s/m!=1:
              if 1000>=N*m*math.log(N*s)>=1 and N*s>=100 and s/m>=100 and 2*math.log(N*s)/math.log(s/m)>1:
                if regime_array[0]==1:
                  if 10**6>N>=10**5:
                   for g in g_array:
                      if s<=0.01:
                        if 5000>=g>=3000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                      else:
                        if g<=2000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                  if 10**7>N>=10**6:
                    for g in g_array:
                      if s<=0.01:
                        if 10000>=g>=8000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                      else:
                        if 2000<g<=4000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])  
                  if 10**8>=N>=10**7:
                    for g in g_array:
                      if s<=0.01:
                        if 15000>=g>=12000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                      else:
                        if 4000<g<=6000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                            
            if s==m:
              if 1000>=N*m*math.log(N*s)>=1 and N*s>=100 and s/m>=100:
                if regime_array[0]==1:
                  if 10**6>N>=10**5:
                   for g in g_array:
                      if s<=0.01:
                        if 5000>=g>=3000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                      else:
                        if g<=2000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                  if 10**7>N>=10**6:
                    for g in g_array:
                      if s<=0.01:
                        if 10000>=g>=8000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                      else:
                        if 2000<g<=4000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])  
                  if 10**8>=N>=10**7:
                    for g in g_array:
                      if s<=0.01:
                        if 15000>=g>=12000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                      else:
                        if 4000<g<=6000:
                            for i in i_array:
                              core_variable_array.append([N,m,s,g,i,'concurrent'])
                            
              if 2*10**(-3)<=N*m*math.log(N*s)<=10**(-1) and N*s>=100 and N<=10**5:
                if regime_array[1]==1:
                  for g in g_array:
                    if g>=3000:
                      for i in i_array:
                        core_variable_array.append([N,m,s,g,i,'successional'])
          
              if 10**(-1)<N*m*math.log(N*s)<1 and N*s>=100:
                if regime_array[2]==1:
                  for g in g_array:
                    for i in i_array:
                      core_variable_array.append([N,m,s,g,i,'between'])
              if N*s<=1:
                if regime_array[3]==1:  
                  if N*m>=10**(-2): # else, trivial result of v ~= 0 
                    for g in g_array:
                      if 25000>=g>=10000:
                        for i in i_array:
                          core_variable_array.append([N,m,s,g,i,'drift'])
            if s==0 and N<=10**3:
              if regime_array[3]==1:  
                  if N*m>=10**(-2): # else, trivial result of v ~= 0 
                    for g in g_array:
                      if 25000>=g>=10000:
                        for i in i_array:
                          core_variable_array.append([N,m,s,g,i,'drift'])
            if s==0 and 10**3<N<=10**4:
              if regime_array[3]==1:  
                  if N*m>=1: # else, trivial result of v ~= 0 
                    for g in g_array:
                      if 25000>=g>=10000:
                        for i in i_array:
                          core_variable_array.append([N,m,s,g,i,'drift']) 
            if s==0 and N>10**4:
              continue

            
  print(core_variable_array)  
  return core_variable_array


n_array=[10**4] 
m_array=[2*10**(-3), 2*10**(-4), 2*10**(-5), 2*10**(-7), 2*10**(-8), 2*10**(-9)]
s_array=[0, 0.002, 0.005, 0.01, 0.02, 0.03, 0.1]
g_array=[1500, 5000, 15000]
g_array=[2000,4000,6000,8000,10000,12000]
i_array=[1] # how many initial mutations existing in the first place?

#core_variable_array=core_variable_array_maker(n_array, m_array, s_array, g_array,i_array, regime_array)


#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################

diverse_s='on' #diverse s interaction to be considered ('on') or not ('off')
par_s=1/math.tan(math.pi*(19/40)) #probability for sampled s from cauchy distribution to be in [-s,s] would be 19/20 from the gamma analysis of cauchy distribution


dynamic_mode='off' # varying N situation to be considered ('on') or not ('off')
N_i=10 # when dynamic mode is 'on', what is the initial total number of population?
grad_var=0.5 # refer to Gompez model for population growth curve
lag_var=100 # until to what generation does the lag phase end?


bound=0.1 # when measuring the fixation, how would you want to set the criteria of proportion for deciding whether certain mutation is fixed or not
alp_condition_array=(0,10**5,2) # when predicting alpha value for the concurrent regime fixation rate, candidates are drawn from (least value, largest value, minimal intervals for prediction) 
alp_array=alp_array_maker(alp_condition_array)
training_size=30 # number of iterations designated for alpha estimation


vel_draw_left_criteria=[5,5,5,6] #setting left sigma limit for velocity histogram of concurrent, successional, between, drift regimes in order
vel_draw_right_criteria=[5,5,5,6] #setting right sigma limit for velocity histogram of concurrent, successional, between, drift regimes in order
fix_draw_left_criteria=[5,5,5,6]  #setting left sigma limit for fixation histogram of concurrent, successional, between, drift regimes in order
fix_draw_right_criteria=[5,5,5,6] #setting right sigma limit for fixation histogram of concurrent, successional, between, drift regimes in order


regime_array=[1,1,1,1]#concurrent, successional, between, drift : here, 1 turns on any particular regime you choose to be analyzed
iteration_size=300 #total iteration number
directory='your directory'
draw_iter_array=[0,0,0,0]# how many iterations for drawing should be done for each of the concurrent, successional, between, drift regime? 0 means 1 drawing, 1 means 2 drawings, and so on


save_switch_array=[0,0,1,'2D',1] 

#save_switch_array[0]: determines whether the total data including every pickle data should be stored(1) or only specific information should be stored(0)

#save_switch_array[1]: do not modify

#save_switch_array[2]: determines whether to save the pickle dictionary data(0) or not(1), recommend not to modify

#in order for save_switch[2],[3],[4] to operate, save_switch[0]==(should be)0

#save_switch_array[3]: 1, 1D, 2D, both are possible selections
#if '1' total data for excel and every figures are made, 
#if 'both' 1D 2D are both drawn for designated iterations from draw_iter_array
#if '1D' only 1D drawn for desginated iterations 
#if '2D' only 2D drawn for designated iterations 

#save_switch_array[4]: if 1 only excel file and velocity graphs are drawn, and 1D, 2D pots are not made


#strongly recommend to only modify the save_switch_array[3] and [4]


core_variable_array='your array' # in the form of [[N1,m1,s1,g1,i1,'regime1'],[N2,m2,s2,g2,i2,'regime2'],...,[Nx,mx,sx,gx,ix,'regimex']]

statistical_info(par_s, diverse_s, draw_iter_array, training_size, alp_array, save_switch_array, iteration_size, core_variable_array, directory, prime_set, prime_set_initial, bound, b_set_size, vel_draw_left_criteria, vel_draw_right_criteria, fix_draw_left_criteria, fix_draw_right_criteria,N_i,grad_var,lag_var,dynamic_mode)
